<a href="https://colab.research.google.com/github/oktaviacitra/data-mining/blob/main/Assignment5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# General

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut

## Upload A File

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Read Dataset

In [ ]:
path = "/content/drive/MyDrive/Data Mining/titanic.csv"
dataset = pd.read_csv(path)
dataset.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Data Normalization

In [ ]:
new_min, new_max = 0, 1
def train_min_max(input_data):
  new_min_max = []
  for i in (0, 1):
    column = input_data.columns[i]
    data = input_data[column]
    min = data.min()
    max = data.max()
    new_min_max.append([min, max])
    input_data[column] = (data - min) * (new_max - new_min) / (max - min) + new_min
  return new_min_max, input_data

In [ ]:
def test_min_max(input_data, new_min_max):
  for i in range(0, 1):
    column = input_data.columns[i]
    data = input_data[column]
    min = new_min_max[i][0]
    max = new_min_max[i][1]
    input_data[column] = (data - min) * (new_max - new_min) / (max - min) + new_min
  return input_data

## K-Nearest Neighbors

In [ ]:
def classification(train_data, train_label, test_data):
  kNN = KNeighborsClassifier(n_neighbors=3, weights='distance')
  kNN.fit(train_data, train_label)
  class_result = kNN.predict(test_data)
  return kNN, class_result

In [ ]:
def error_ratio(knn, test_data, test_label):
  precision_ratio = knn.score(test_data, test_label)
  error_ratio = 1 - precision_ratio
  return error_ratio

# Validation Model of Classification

## Holdout Method

In [ ]:
train_data = dataset.drop('Survived', axis=1)
label = dataset.loc[:, ['Survived']]
ho_train_data, ho_test_data, ho_train_label, ho_test_label = train_test_split(train_data, label, train_size=0.7)
ho_train_data = ho_train_data.loc[:, ['Age', 'Fare', 'Pclass', 'Sex']].fillna(ho_train_data.mean())
ho_test_data = ho_test_data.loc[:, ['Age', 'Fare', 'Pclass', 'Sex']].fillna(ho_test_data.mean())
ho_train_data['Sex'] = ho_train_data['Sex'].replace({'female':1, 'male':0})
ho_test_data['Sex'] = ho_test_data['Sex'].replace({'female':1, 'male':0})
ho_new_min_max, ho_train_data = train_min_max(ho_train_data)
ho_test_data = test_min_max(ho_test_data, ho_new_min_max)
ho_knn, ho_result = classification(ho_train_data, ho_train_label, ho_test_data)
ho_error_ratio = error_ratio(ho_knn, ho_test_data, ho_test_label)
print("Holdout Error Ratio : " + str(ho_error_ratio))

Holdout Error Ratio : 0.5970149253731343


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


## K-Fold Cross Validation

In [ ]:
n=20
kfold = KFold(n_splits=10)
final_kfold_error_ratio = 0
for train_index, test_index in kfold.split(dataset):
  kfold_train_data, kfold_test_data = dataset.iloc[train_index], dataset.iloc[test_index]
  kfold_train_label = kfold_train_data.loc[:, ['Survived']]
  kfold_test_label = kfold_test_data.loc[:, ['Survived']]
  kfold_train_data = kfold_train_data.loc[:, ['Age', 'Fare', 'Pclass', 'Sex']].fillna(kfold_train_data.mean())
  kfold_test_data = kfold_test_data.loc[:, ['Age', 'Fare', 'Pclass', 'Sex']]
  kfold_train_data['Sex'] = kfold_train_data['Sex'].replace({'female':1, 'male':0})
  kfold_test_data['Sex'] = kfold_test_data['Sex'].replace({'female':1, 'male':0})
  kfold_new_min_max, kfold_train_data = train_min_max(kfold_train_data)
  kfold_test_data = test_min_max(kfold_test_data, kfold_new_min_max)
  # pos_missing = [index for index, row in kfold_test_data.iterrows() if row.isnull().any()]
  iter = 0
  pos_missing = []
  for index, row in kfold_test_data.iterrows():
    if row.isnull().any():
      pos_missing.append(iter)
    iter += 1
  kfold_test_data_temp = dataset.iloc[test_index].dropna()
  kfold_test_data_full = kfold_test_data_temp.loc[:, ['Age', 'Fare', 'Pclass', 'Sex']].replace({'female':1, 'male':0})
  kfold_test_label_full = kfold_test_data_temp.loc[:, ['Survived']]
  kfold_knn, kfold_result = classification(kfold_train_data, kfold_train_label, kfold_test_data_full)
  kfold_eror_ratio_full = error_ratio(kfold_knn, kfold_test_data_full, kfold_test_label_full)
  kfold_train_data_miss = kfold_train_data.loc[:, ['Fare', 'Pclass', 'Sex']]
  kfold_test_data_miss = kfold_test_data.loc[:, ['Fare', 'Pclass', 'Sex']].iloc[pos_missing]
  kfold_test_label_miss = kfold_test_label.iloc[pos_missing]
  kfold_knn, kfold_result = classification(kfold_train_data_miss, kfold_train_label, kfold_test_data_miss)
  kfold_eror_ratio_miss = error_ratio(kfold_knn, kfold_test_data_miss, kfold_test_label_miss)
  final_kfold_error_ratio += (kfold_eror_ratio_miss + kfold_eror_ratio_full)
print("K-Fold Error Ratio : " + str(final_kfold_error_ratio / n))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A colum

K-Fold Error Ratio : 0.48826719621232684


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


## Leave-one-out Cross Validation

In [ ]:
loo = LeaveOneOut()
final_loo_error_ratio = 0
for train_index, test_index in loo.split(dataset):
  loo_train_data, loo_test_data = dataset.iloc[train_index], dataset.iloc[test_index]
  loo_train_label = loo_train_data.loc[:, ['Survived']]
  loo_test_label = loo_test_data.loc[:, ['Survived']]
  loo_train_data = loo_train_data.loc[:, ['Age', 'Fare', 'Pclass', 'Sex']].fillna(loo_train_data.mean())
  loo_test_data = loo_test_data.loc[:, ['Age', 'Fare', 'Pclass', 'Sex']]
  loo_train_data['Sex'] = loo_train_data['Sex'].replace({'female':1, 'male':0})
  loo_test_data['Sex'] = loo_test_data['Sex'].replace({'female':1, 'male':0})
  if loo_test_data['Age'].isnull().values.any() == True:
    loo_test_data = loo_test_data.loc[:, ['Fare', 'Pclass', 'Sex']]
    loo_train_data = loo_train_data.loc[:, ['Fare', 'Pclass', 'Sex']]
  loo_new_min_max, loo_train_data = train_min_max(loo_train_data)
  loo_test_data = test_min_max(loo_test_data, loo_new_min_max)
  loo_knn, loo_result = classification(loo_train_data, loo_train_label, loo_test_data)
  loo_error_ratio = error_ratio(loo_knn, loo_test_data, loo_test_label)
  final_loo_error_ratio += loo_error_ratio
print("Leave-one-out Error Ratio : " + str(final_loo_error_ratio/891))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A colum

Leave-one-out Error Ratio : 0.5117845117845118


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
